In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = (
  SparkSession.builder
    .appName("AirlineDemo")
    .enableHiveSupport()
    .getOrCreate()
)

25/05/05 14:58:44 WARN Utils: Your hostname, konkuterWK resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/05/05 14:58:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/05 14:58:44 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [16]:
from pyspark.sql.functions import col, floor, avg, count, when

 # 2) Load the Hive table, filter out cancelled flights
df = (spark.table("default.airline_data")
            .filter(col("Cancelled") == 0))

# 3) Derive an integer departure hour (0–23)
#     (DepTime is hhmm; e.g. 530 → 5, 1745 → 17)
df = df.withColumn("DepHour", floor(col("DepTime") / 100))

# 4) Compute average arrival delay by hour, day of week, and month
hour_stats = (df.groupBy("DepHour")
                .agg(
                    avg(col("ArrDelay")).alias("avg_arr_delay"),
                    count("*").alias("num_flights")
                )
                .orderBy("avg_arr_delay"))

dow_stats = (df.groupBy("DayOfWeek")
                .agg(
                    avg(col("ArrDelay")).alias("avg_arr_delay"),
                    count("*").alias("num_flights")
                )
                .orderBy("avg_arr_delay"))

month_stats = (df.groupBy("Month")
                    .agg(
                    avg(col("ArrDelay")).alias("avg_arr_delay"),
                    count("*").alias("num_flights")
                    )
                    .orderBy("avg_arr_delay"))

# 5) Optionally: bucket hours into periods of day
df = df.withColumn(
    "DepPeriod",
    when(col("DepHour").between(5, 8), "Early Morning")
    .when(col("DepHour").between(9, 11), "Morning")
    .when(col("DepHour").between(12, 15), "Afternoon")
    .when(col("DepHour").between(16, 18), "Evening")
    .when(col("DepHour").between(19, 22), "Night")
    .otherwise("Late Night")
)
period_stats = (df.groupBy("DepPeriod")
                    .agg(
                    avg(col("ArrDelay")).alias("avg_arr_delay"),
                    count("*").alias("num_flights")
                    )
                    .orderBy("avg_arr_delay"))

# 6) Show the top-5 best buckets in each dimension
print("\n=== Best Hours to Fly (by avg arrival delay) ===")
hour_stats.show(5, truncate=False)

print("\n=== Best Days of Week to Fly ===")
dow_stats.show(7, truncate=False)

print("\n=== Best Months to Fly ===")
month_stats.show(12, truncate=False)

print("\n=== Best Periods of Day to Fly ===")
period_stats.show(truncate=False)


=== Best Hours to Fly (by avg arrival delay) ===


+-------+-------------------+-----------+
|DepHour|avg_arr_delay      |num_flights|
+-------+-------------------+-----------+
|5      |-4.234302845615189 |1197431    |
|6      |-1.5800270771518068|7097574    |
|7      |0.47752091410698677|8103521    |
|8      |1.5478352314658903 |8441451    |
|9      |3.233647040049947  |7748592    |
+-------+-------------------+-----------+
only showing top 5 rows


=== Best Days of Week to Fly ===


+---------+------------------+-----------+
|DayOfWeek|avg_arr_delay     |num_flights|
+---------+------------------+-----------+
|6        |4.187418958786977 |15652251   |
|2        |5.960420592346179 |17685384   |
|7        |6.525039865667659 |16885337   |
|1        |6.6695148496840915|17791603   |
|3        |7.091501517465088 |17739726   |
|4        |8.945046639782811 |17726892   |
|5        |9.606953425895007 |17750452   |
+---------+------------------+-----------+


=== Best Months to Fly ===


+-----+------------------+-----------+
|Month|avg_arr_delay     |num_flights|
+-----+------------------+-----------+
|9    |3.415275403771856 |9710893    |
|10   |4.757955359226459 |10624270   |
|4    |5.253459694828195 |9956265    |
|11   |5.285870766188475 |10092511   |
|5    |5.484669713931747 |10203649   |
|3    |7.24424920498482  |10235777   |
|8    |7.760446570820322 |10480041   |
|2    |7.890012028806683 |9182556    |
|1    |8.429739020654077 |9950241    |
|7    |8.84406501360377  |10405418   |
|6    |9.703535549331425 |10060951   |
|12   |10.455688276350626|10329073   |
+-----+------------------+-----------+


=== Best Periods of Day to Fly ===


+-------------+-------------------+-----------+
|DepPeriod    |avg_arr_delay      |num_flights|
+-------------+-------------------+-----------+
|Early Morning|0.02626013117114535|24839977   |
|Morning      |3.8790210373277816 |22448465   |
|Afternoon    |6.430646504853383  |30230542   |
|Evening      |9.455105394251209  |22941086   |
|Night        |15.561504282002353 |18981670   |
|Late Night   |33.77991295742121  |1789905    |
+-------------+-------------------+-----------+



In [ ]:
from pyspark.sql.functions import min
df = (spark.table("default.airline_data")
             .filter((col("Cancelled") == 0) & col("TailNum").isNotNull()))

    # 2) Compute first year seen per aircraft
first_year = (df.select("TailNum", "Year")
                .groupBy("TailNum")
                .agg(min("Year").alias("FirstYear")))

# 3) Join back to get Age
df_age = (df.join(first_year, on="TailNum")
            .withColumn("Age", col("Year") - col("FirstYear")))

# 4) Compute average arrival delay by Age
age_stats = (df_age.groupBy("Age")
                    .agg(
                    avg(col("ArrDelay")).alias("avg_arr_delay"),
                    count("*").alias("num_flights")
                    )
                    .orderBy("Age"))

# 5) Show results (you can filter out very high ages or low-flight counts)
age_stats.show(truncate=False)

25/05/05 15:37:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 15:37:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 15:37:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 15:37:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 15:37:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 15:37:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 15:37:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 15:37:09 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
25/05/05 15:37:09 WARN RowBasedKeyValueBatch: Calling spill() on

+---+------------------+-----------+
|Age|avg_arr_delay     |num_flights|
+---+------------------+-----------+
|0  |6.483918377240217 |18240802   |
|1  |7.056773405765787 |18632789   |
|2  |7.842096954720509 |15481942   |
|3  |7.491527738595653 |14440076   |
|4  |6.933445866562948 |13464447   |
|5  |7.571370266220807 |11872967   |
|6  |5.494010546853433 |6661247    |
|7  |5.233546474885201 |8582379    |
|8  |4.585900008699769 |3432106    |
|9  |7.680641826983231 |2883330    |
|10 |8.208322857575762 |2407552    |
|11 |9.202039183471316 |2015720    |
|12 |11.945164911389554|1673863    |
|13 |9.978788019916294 |1442425    |
+---+------------------+-----------+



25/05/05 17:28:34 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 633972 ms exceeds timeout 120000 ms
25/05/05 17:28:34 WARN SparkContext: Killing executors is not supported by current scheduler.
25/05/05 17:28:37 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:322)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:117)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$driverEndpoint(BlockManagerMasterEndpoint.scala:116)
	at org.apache.spark.storage.B